In [1]:
import numpy as np
from random import shuffle

from vigenere_utils import *
import pandas as pd
import shutil, os

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.contrib import rnn, seq2seq

In [2]:
show_result('madlabrules', 'deeplearning')

Key: deeplearning
Plaintext: madlabrules
Encrytped: pehalfrlymf
Decrytped: madlabrules


In [3]:
logs_path = '/home/user/tensorflow_logs/vigenere_LSTM'
key = "madlab"
seq_length = 15

In [4]:
def get_random_sequence(ALPHA, seq_length):
        seq = ''.join(np.random.choice(a=list(ALPHA), replace=True, size=seq_length))
        return seq

In [5]:
def one_hot_encode(sequence, n_unique=len(ALPHA)):
	encoding = list()
	for value in sequence:
		vector = [0 for _ in range(n_unique)]
		vector[ALPHA.index(value)] = 1
		encoding.append(vector)
	return np.array(encoding)

In [7]:
np.random.seed(1616)
original_sequences = [get_random_sequence(ALPHA, seq_length) for i in range(2**20)]
np.random.seed(1616)
shuffle(original_sequences)

In [8]:
encrypted_sequences = [encrypt(key=key, plaintext=s) for s in original_sequences]

In [9]:
original_sequences_oh = [one_hot_encode(s) for s in original_sequences]
encrypted_sequences_oh = [one_hot_encode(s) for s in encrypted_sequences]
original_argmax = []
encrypted_argmax = []

In [10]:
for s in original_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    original_argmax.append(t)

In [11]:
for s in encrypted_sequences_oh:
    t = []
    for c in s:
        t.extend([np.argmax(c)])
    encrypted_argmax.append(t)

In [12]:
batch_size = 10000
num_steps = seq_length
num_classes = len(ALPHA)
state_size = 256
learning_rate = 0.01
tf.reset_default_graph()

In [13]:
x = tf.placeholder(tf.int32, [batch_size, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size, state_size])

rnn_inputs = tf.one_hot(x, num_classes)

In [14]:
cell = tf.contrib.rnn.LSTMCell(state_size, state_is_tuple=True)

In [15]:
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, rnn_inputs, dtype=tf.float32)

In [16]:
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = tf.reshape(
            tf.matmul(tf.reshape(rnn_outputs, [-1, state_size]), W) + b,
            [batch_size, num_steps, num_classes])
predictions = tf.nn.softmax(logits)


In [17]:
losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss)

In [18]:
tf.summary.scalar("error", total_loss)
# Create a summary to monitor accuracy tensor
# Create summaries to visualize weights
for var in tf.trainable_variables():
    tf.summary.histogram(var.name, var)
    
merged_summary_op = tf.summary.merge_all()

INFO:tensorflow:Summary name rnn/lstm_cell/kernel:0 is illegal; using rnn/lstm_cell/kernel_0 instead.
INFO:tensorflow:Summary name rnn/lstm_cell/bias:0 is illegal; using rnn/lstm_cell/bias_0 instead.
INFO:tensorflow:Summary name softmax/W:0 is illegal; using softmax/W_0 instead.
INFO:tensorflow:Summary name softmax/b:0 is illegal; using softmax/b_0 instead.


In [19]:
def generate_batch(encrypted, original, batch_size):
    n_batches = len(encrypted)/batch_size
    for n in range(int(n_batches)):
        if((n+1)*batch_size+1 < len(encrypted_argmax)):
            yield(encrypted_argmax[n*batch_size:(n+1)*batch_size],\
                 original_argmax[n*batch_size:(n+1)*batch_size])
        else:
            yield(encrypted_argmax[n*batch_size:], \
                 original_argmax[n*batch_size:])

def gen_epoch(encrypted, original, n_epochs, batch_size):
    for e in range(n_epochs):
        yield(generate_batch(encrypted, original, batch_size))

In [20]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

In [21]:
if(os.path.exists(logs_path)):
    print("Removing logs")
    shutil.rmtree(logs_path)

sess = tf.InteractiveSession()

sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter(logs_path,
                                            graph=tf.get_default_graph())

training_state = np.zeros((batch_size, state_size))

st=-1
for j, gen in enumerate(gen_epoch(encrypted_argmax, original_argmax, 5, batch_size)):
    print("Epoch")
    for i, (X, Y) in enumerate(gen):
        st+=1
        tr_losses, training_loss_, training_state, _, p, summary = \
                            sess.run([losses,
                                      total_loss,
                                      final_state,
                                      train_step,
                                     logits,
                                     merged_summary_op],
                                          feed_dict={x:X, y:Y})
        summary_writer.add_summary(summary, st)
        if(i % 1 == 0):
            print("Epoch: "+"{:.0f}".format(j)+" - Batch: "+"{:.0f}".format(i)+" - loss: "+"{:f}".format(training_loss_))
            print("input: "+''.join([ALPHA[i] for i in X[0]]))
            print("target: "+''.join([ALPHA[i] for i in Y[0]]))
            print("pred: "+''.join([ALPHA[np.argmax(i)] for i in p[0]]))

Epoch
Epoch: 0 - Batch: 0 - loss: 3.259946
input: bagszotftbavyjd
target: padhznhfqqaumja
pred: mtnkkkgxxmxttxp
Epoch: 0 - Batch: 1 - loss: 3.232537
input: kiufhnfnvsmdmcg
target: yiruhmtnshmcacd
pred: hhbfhhffvvddddd
Epoch: 0 - Batch: 2 - loss: 3.202883
input: acuntkqxpqxnbjh
target: ocrctjexmfxmpje
pred: aqunqkqqqqqqqqq
Epoch: 0 - Batch: 3 - loss: 3.170847
input: utztprlxhrxejzt
target: itwipqzxegxdxzq
pred: utzttrrxhrrbggs
Epoch: 0 - Batch: 4 - loss: 3.124503
input: wjwvgullgjjiioj
target: kjtkgtzldyjhwog
pred: wjwwwuulugjjiuj
Epoch: 0 - Batch: 5 - loss: 3.216875
input: doytwlvrqlpbdjb
target: roviwkjrnaparjy
pred: doyttlvfffvvvvv
Epoch: 0 - Batch: 6 - loss: 3.061912
input: ytjaneqavektwok
target: mtgpndeastkskoh
pred: ytjxnbqaabkttlz
Epoch: 0 - Batch: 7 - loss: 3.050430
input: iviaagqtnpzctlf
target: wvfpafetkezbhlc
pred: iviaagqqnpnztzt
Epoch: 0 - Batch: 8 - loss: 3.031065
input: ohblchdtzjehvme
target: chyacgrtwyegjmb
pred: ohblchdhzjehvje
Epoch: 0 - Batch: 9 - loss: 3.004200
inp

Epoch: 0 - Batch: 78 - loss: 0.431232
input: lhyulqapochgvek
target: zhvjlpoplrhfjeh
pred: zhvjlpoplrhfjeh
Epoch: 0 - Batch: 79 - loss: 0.386502
input: pgfvynfeevawnuz
target: dgckymtebkavbuw
pred: dgckymtebkavbuw
Epoch: 0 - Batch: 80 - loss: 0.344220
input: zgyosrgzmxyabku
target: ngvdsquzjmyzpkr
pred: ngvdsquzjmyzpkr
Epoch: 0 - Batch: 81 - loss: 0.302690
input: louugkxdkypigyj
target: zorjgjldhnphuyg
pred: zorjgjldhnphuyg
Epoch: 0 - Batch: 82 - loss: 0.268928
input: oyzuzxhnstdqzdk
target: cywjzwvnpidpndh
pred: cywjzwvnpidpndh
Epoch: 0 - Batch: 83 - loss: 0.239852
input: pruahbwwlmlwtgn
target: drrphakwiblvhgk
pred: drrphakwiblvhgk
Epoch: 0 - Batch: 84 - loss: 0.212823
input: zoejdyrwhffhnuo
target: nobydxfweufgbul
pred: nobydxfweufgbul
Epoch: 0 - Batch: 85 - loss: 0.187986
input: popyjlufkyflxya
target: domnjkifhnfklyx
pred: domnjkifhnfklyx
Epoch: 0 - Batch: 86 - loss: 0.166616
input: nyuusavznyxmpzn
target: byrjszjzknxldzk
pred: byrjszjzknxldzk
Epoch: 0 - Batch: 87 - loss: 0.147438

Epoch: 1 - Batch: 51 - loss: 0.006671
input: naseghhzngjooin
target: baptggvzkvjncik
pred: baptggvzkvjncik
Epoch: 1 - Batch: 52 - loss: 0.006562
input: agzicarxklaiozv
target: ogwxczfxhaahczs
pred: ogwxczfxhaahczs
Epoch: 1 - Batch: 53 - loss: 0.006487
input: zoaqlsosefdeijl
target: noxflrcsbuddwji
pred: noxflrcsbuddwji
Epoch: 1 - Batch: 54 - loss: 0.006397
input: jbuefukgyzajkuf
target: xbrtftygvoaiyuc
pred: xbrtftygvoaiyuc
Epoch: 1 - Batch: 55 - loss: 0.006288
input: mawvmzbzupzccej
target: aatkmypzrezbqeg
pred: aatkmypzrezbqeg
Epoch: 1 - Batch: 56 - loss: 0.006221
input: ejxulxzvvhifdfp
target: sjujlwnvswierfm
pred: sjujlwnvswierfm
Epoch: 1 - Batch: 57 - loss: 0.006124
input: sqsjkfoxomknlal
target: gqpykecxlbkmzai
pred: gqpykecxlbkmzai
Epoch: 1 - Batch: 58 - loss: 0.006050
input: zaqwjkconjwykrl
target: nanljjqokywxyri
pred: nanljjqokywxyri
Epoch: 1 - Batch: 59 - loss: 0.005966
input: keonqlgnygzqvva
target: yelcqkunvvzpjvx
pred: yelcqkunvvzpjvx
Epoch: 1 - Batch: 60 - loss: 0.005908

Epoch: 2 - Batch: 25 - loss: 0.002943
input: msknmsbedrriwhe
target: ashcmrpeagrhkhb
pred: ashcmrpeagrhkhb
Epoch: 2 - Batch: 26 - loss: 0.002920
input: fnqsppxemfsztof
target: tnnhpolejusyhoc
pred: tnnhpolejusyhoc
Epoch: 2 - Batch: 27 - loss: 0.002896
input: wnhekmifgymteol
target: knetklwfdnmssoi
pred: knetklwfdnmssoi
Epoch: 2 - Batch: 28 - loss: 0.002874
input: rnnlpqzypljeadd
target: fnkappnymajdoda
pred: fnkappnymajdoda
Epoch: 2 - Batch: 29 - loss: 0.002845
input: ynozyzumtarevjz
target: mnloyyimqprdjjw
pred: mnloyyimqprdjjw
Epoch: 2 - Batch: 30 - loss: 0.002824
input: gzroewsrqzzsvnj
target: uzodevgrnozrjng
pred: uzodevgrnozrjng
Epoch: 2 - Batch: 31 - loss: 0.002806
input: cieqyznnjfxmhgb
target: qibfyybnguxlvgy
pred: qibfyybnguxlvgy
Epoch: 2 - Batch: 32 - loss: 0.002781
input: owjcrtfaujmqewf
target: cwgrrstarympswc
pred: cwgrrstarympswc
Epoch: 2 - Batch: 33 - loss: 0.002756
input: dttcefawwdmxzmn
target: rtqreeowtsmwnmk
pred: rtqreeowtsmwnmk
Epoch: 2 - Batch: 34 - loss: 0.002742

Epoch: 2 - Batch: 103 - loss: 0.001721
input: ltyuxdfuxwoiaen
target: ztvjxctuulohoek
pred: ztvjxctuulohoek
Epoch
Epoch: 3 - Batch: 0 - loss: 0.001712
input: bagszotftbavyjd
target: padhznhfqqaumja
pred: padhznhfqqaumja
Epoch: 3 - Batch: 1 - loss: 0.001703
input: kiufhnfnvsmdmcg
target: yiruhmtnshmcacd
pred: yiruhmtnshmcacd
Epoch: 3 - Batch: 2 - loss: 0.001692
input: acuntkqxpqxnbjh
target: ocrctjexmfxmpje
pred: ocrctjexmfxmpje
Epoch: 3 - Batch: 3 - loss: 0.001684
input: utztprlxhrxejzt
target: itwipqzxegxdxzq
pred: itwipqzxegxdxzq
Epoch: 3 - Batch: 4 - loss: 0.001671
input: wjwvgullgjjiioj
target: kjtkgtzldyjhwog
pred: kjtkgtzldyjhwog
Epoch: 3 - Batch: 5 - loss: 0.001665
input: doytwlvrqlpbdjb
target: roviwkjrnaparjy
pred: roviwkjrnaparjy
Epoch: 3 - Batch: 6 - loss: 0.001653
input: ytjaneqavektwok
target: mtgpndeastkskoh
pred: mtgpndeastkskoh
Epoch: 3 - Batch: 7 - loss: 0.001645
input: iviaagqtnpzctlf
target: wvfpafetkezbhlc
pred: wvfpafetkezbhlc
Epoch: 3 - Batch: 8 - loss: 0.001633
i

Epoch: 3 - Batch: 76 - loss: 0.001147
input: vxllklxlvyyahpt
target: jxiakkllsnyzvpq
pred: jxiakkllsnyzvpq
Epoch: 3 - Batch: 77 - loss: 0.001145
input: cwwzsbaqfrpacgy
target: qwtosaoqcgpzqgv
pred: qwtosaoqcgpzqgv
Epoch: 3 - Batch: 78 - loss: 0.001142
input: lhyulqapochgvek
target: zhvjlpoplrhfjeh
pred: zhvjlpoplrhfjeh
Epoch: 3 - Batch: 79 - loss: 0.001134
input: pgfvynfeevawnuz
target: dgckymtebkavbuw
pred: dgckymtebkavbuw
Epoch: 3 - Batch: 80 - loss: 0.001129
input: zgyosrgzmxyabku
target: ngvdsquzjmyzpkr
pred: ngvdsquzjmyzpkr
Epoch: 3 - Batch: 81 - loss: 0.001124
input: louugkxdkypigyj
target: zorjgjldhnphuyg
pred: zorjgjldhnphuyg
Epoch: 3 - Batch: 82 - loss: 0.001120
input: oyzuzxhnstdqzdk
target: cywjzwvnpidpndh
pred: cywjzwvnpidpndh
Epoch: 3 - Batch: 83 - loss: 0.001114
input: pruahbwwlmlwtgn
target: drrphakwiblvhgk
pred: drrphakwiblvhgk
Epoch: 3 - Batch: 84 - loss: 0.001111
input: zoejdyrwhffhnuo
target: nobydxfweufgbul
pred: nobydxfweufgbul
Epoch: 3 - Batch: 85 - loss: 0.001105

Epoch: 4 - Batch: 50 - loss: 0.000825
input: rjfwldkrtvuzjaq
target: fjcllcyrqkuyxan
pred: fjcllcyrqkuyxan
Epoch: 4 - Batch: 51 - loss: 0.000823
input: naseghhzngjooin
target: baptggvzkvjncik
pred: baptggvzkvjncik
Epoch: 4 - Batch: 52 - loss: 0.000818
input: agzicarxklaiozv
target: ogwxczfxhaahczs
pred: ogwxczfxhaahczs
Epoch: 4 - Batch: 53 - loss: 0.000817
input: zoaqlsosefdeijl
target: noxflrcsbuddwji
pred: noxflrcsbuddwji
Epoch: 4 - Batch: 54 - loss: 0.000815
input: jbuefukgyzajkuf
target: xbrtftygvoaiyuc
pred: xbrtftygvoaiyuc
Epoch: 4 - Batch: 55 - loss: 0.000809
input: mawvmzbzupzccej
target: aatkmypzrezbqeg
pred: aatkmypzrezbqeg
Epoch: 4 - Batch: 56 - loss: 0.000808
input: ejxulxzvvhifdfp
target: sjujlwnvswierfm
pred: sjujlwnvswierfm
Epoch: 4 - Batch: 57 - loss: 0.000804
input: sqsjkfoxomknlal
target: gqpykecxlbkmzai
pred: gqpykecxlbkmzai
Epoch: 4 - Batch: 58 - loss: 0.000802
input: zaqwjkconjwykrl
target: nanljjqokywxyri
pred: nanljjqokywxyri
Epoch: 4 - Batch: 59 - loss: 0.000798

In [28]:
print("input: "+''.join([ALPHA[i] for i in X[0]]))
print("target: "+''.join([ALPHA[i] for i in Y[0]]))
print("pred: "+''.join([ALPHA[np.argmax(i)] for i in p[0]]))
print("pred_enc: "+encrypt(key, ''.join([ALPHA[np.argmax(i)] for i in p[0]])))

input: ltyuxdfuxwoiaen
target: ztvjxctuulohoek
pred: ztvjxctuulohoek
pred_enc: ltyuxdfuxwoiaen


In [23]:
print(encrypted_sequences[i*batch_size])
print(original_sequences[i*batch_size])
print(encrypt(key, original_sequences[i*batch_size]))

ltyuxdfuxwoiaen
ztvjxctuulohoek
ltyuxdfuxwoiaen
